# APIM ❤️ AI Agents

## Azure AI Agent Service lab - (AI Foundry - Projects New)

![flow](../../images/ai-agent-service.gif)

Use this playground to explore the [Azure AI Agent Service](https://learn.microsoft.com/en-us/azure/ai-services/agents/overview), leveraging Azure API Management to control multiple services, including Logic Apps Workflows, and OpenAPI-based APIs. This enables limitless opportunities for AI agents while maintaining control through Azure API Management!

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the models and versions according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}-foundry-v2" # change the name to match your naming style
resource_group_location = "eastus2"

aiservices_config = [{"name": "foundry1", "location": "eastus2"}]
models_config = [{"name": "gpt-4.1-mini", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 20}]

apim_sku = 'Basicv2'
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

inference_api_path = "inference"  # path to the inference API in the APIM service
inference_api_type = "AzureAI"  # options: AzureOpenAI, AzureAI, OpenAI, PassThrough
inference_api_version = "2024-05-01-preview"
foundry_project_name = deployment_name

utils.print_ok('Notebook initialized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "aiServicesConfig": { "value": aiservices_config },
        "modelsConfig": { "value": models_config },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "inferenceAPIPath": { "value": inference_api_path },
        "inferenceAPIType": { "value": inference_api_type },
        "foundryProjectName": { "value": foundry_project_name }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main-v2.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for subscription in apim_subscriptions:
        subscription_name = subscription['name']
        subscription_key = subscription['key']
        utils.print_info(f"Subscription Name: {subscription_name}")
        utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
    api_key = apim_subscriptions[0].get("key") # default api key to the first subscription key
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    foundry_project_endpoint = utils.get_deployment_output(output, 'foundryProjectEndpoint', 'Foundry Project Endpoint')


<a id='4'></a>
### 4️⃣ List the connections

Retrieve the connections managed in AI Foundry available for the Agents

In [ ]:
%pip install --upgrade azure-ai-projects>=2.0.0b3

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(credential=DefaultAzureCredential(),
    endpoint=foundry_project_endpoint)
with project_client:
    connections = project_client.connections.list()
    for connection in connections:
        utils.print_info(f"Name: {connection.name}, Id: {connection.id}, Type: {connection.type}")



<a id='quickstart'></a>
### 🧪 Create and run a Math Tutor Agent with AI Foundry Agents API

Check the official documentation for updates on this quickstart:
https://learn.microsoft.com/en-us/azure/ai-services/agents/quickstart

***Important: This cell is prone to high latency due to the adhoc nature of creating a codeinterpreter container - 3-5 mins***

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import PromptAgentDefinition, CodeInterpreterTool, CodeInterpreterToolAuto


# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)
agents_client = project_client.agents

# Initialize the Code Interpreter tool with auto container and no pre-uploaded files
code_interpreter = CodeInterpreterTool(container=CodeInterpreterToolAuto(file_ids=[]))

with project_client:
    agent = agents_client.create_version(
        definition=PromptAgentDefinition(
            model=str(models_config[0].get('name')),
            instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
            tools=[code_interpreter]
        ),
        agent_name="my-maths-agent"
    )
    utils.print_ok(f"Created agent, ID: {agent.id}")

    openai_client = project_client.get_openai_client()
    conversation_id = openai_client.conversations.create().id
    utils.print_ok(f"Created thread, ID: {conversation_id}")
    
    # Create message to thread
    response = openai_client.responses.create(
        input=[{"role": "user", "content": "I need to solve the equation `3x + 11 = 14`. Can you help me?"}],
        conversation=conversation_id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )
    
    # Fetch and log all messages
    print(f"\n🗨️ Agent: {response.output_text}")
    
    ### Clean up resources
    # agents_client.delete_version(agent_name=agent.name, agent_version=agent.version)
    # utils.print_ok("Deleted agent")

<a id='bing'></a>
### 🧪 Grounding with Bing

Check the official documentation for updates on this sample: https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder, MessageTextContent
from azure.ai.projects.models import PromptAgentDefinition, BingGroundingAgentTool, BingGroundingSearchToolParameters, BingGroundingSearchConfiguration

prompt_content = "What are the top 5 news headlines today from the UK?"

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)
agents_client = project_client.agents

bing_connection = project_client.connections.get(name='bingSearch-connection')
conn_id = bing_connection.id

# Initialize agent bing tool and add the connection id

bing = BingGroundingAgentTool(
        bing_grounding=BingGroundingSearchToolParameters(
            search_configurations=[
                BingGroundingSearchConfiguration(
                    project_connection_id=conn_id,
                    market="en-gb",             # specify market for localized results, e.g. "en-GB" for UK
                    set_lang="en",
                    count=5
                )
            ]
        )
    )

# Create agent with the bing tool and process assistant run
with project_client:
    bing_agent = agents_client.create_version(
        definition=PromptAgentDefinition(
            model=str(models_config[0].get('name')),
            instructions="Provide concise and accurate answers based on the search results in bullet points for easy reading.",
            tools=[bing]
        ),
        agent_name="my-bing-assistant",
        headers={"x-ms-enable-preview": "true"})   
    utils.print_ok(f"Created agent, ID: {bing_agent.id}")

    openai_client = project_client.get_openai_client()
    conversation_id = openai_client.conversations.create().id
    utils.print_ok(f"Created thread, ID: {conversation_id}")

    # Create message to thread
    response = openai_client.responses.create(
        input=[{"role": "user", "content": "What's the latest news from the UK?"}],
        conversation=conversation_id,
        extra_body={"agent": {"name": bing_agent.name, "type": "agent_reference"}},
    )   

    # Fetch and log all messages
    print(f"\n🗨️ Agent: {response.output_text}")

    ### Clean up resources
    # agents_client.delete_version(agent_name=bing_agent.name, agent_version=bing_agent.version)
    # utils.print_ok("Deleted agent")

<a id='weatherapi'></a>
### 🧪 Run agent with Weather OpenAPI from Azure API Management

👉 Check the [Azure AI Foundry Tracing](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/trace) information to understand the execution process.


In [ ]:
import json
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import PromptAgentDefinition


credential=DefaultAzureCredential()
project_client=AIProjectClient(endpoint=foundry_project_endpoint, credential=credential)
openai_client=project_client.get_openai_client()

prompt_content = "Return a summary of the temperature in Seattle and 3 other sister cities in Europe?"

# Load OpenAPI specification for weather API
with open("./city-weather-openapi.json", "r") as f:
    openapi_weather = json.loads(f.read().replace("https://replace-me.local/weatherservice", f"{apim_resource_gateway_url}/weatherservice"))

# Initialize agent OpenAPI tool using dictionary-based definition
weather_tool = {
    "type": "openapi",
    "openapi": {
        "name": "get_weather",
        "description": "Retrieve weather information for a location",
        "spec": openapi_weather,
        "auth": {
            "type": "project_connection",
            "security_scheme": {
                "project_connection_id": project_client.connections.get("WeatherAPI").id
            }
        }
    }
}

# Create agent with chained context managers
weather_agent = project_client.agents.create_version(
    agent_name="my-weather-agent",
    definition=PromptAgentDefinition(
        model=str(models_config[0].get('name')),
        instructions="You are a helpful agent that provides weather information for cities around the world.",
        tools=[weather_tool],
    ),
    description="Weather information assistant",
)
utils.print_ok(f"Created agent (id: {weather_agent.id}, name: {weather_agent.name}, version: {weather_agent.version})")

conversation_id = openai_client.conversations.create().id
utils.print_ok(f"Created conversation, ID: {conversation_id}")

print(f"\n🗨️ User: {prompt_content}")
response = openai_client.responses.create(
    input=[{"role": "user", "content": prompt_content}],
    conversation=conversation_id,
    extra_body={"agent": {"name": weather_agent.name, "type": "agent_reference"}},
)
print(f"\n🗨️ Agent: {response.output_text}")

# print("\nCleaning up...")
# project_client.agents.delete_version(agent_name=weather_agent.name, agent_version=weather_agent.version)
# utils.print_ok("Agent deleted")

<a id='logicapp'></a>
### 🧪 Run agent operations with MCP Backend and Logic Apps workflow

⚙️ **Tools**:
- Get Product Catalog - OpenAPI Backend mocked with an APIM policy.
- Place Order - A Logic Apps workflow that processes orders with a maximum of five items.

✨ **Expected Behavior**:
- The agent receives a user request to order 11 smartphones.
- The agent calls the product catalog API to retrieve the product SKU and available stock quantity.
- If the order quantity exceeds available stock, the agent will respond that the order cannot be processed due to insufficient stock.
- If stock is available, the agent will initiate the order workflow, which will fail because the quantity exceeds the maximum limit of five items.
- As the agent was instructed to recover from errors, it will place multiple orders, each with a quantity below the maximum limit, ensuring the total equals the desired order quantity.


In [ ]:
import json
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import PromptAgentDefinition

credential=DefaultAzureCredential()
project_client=AIProjectClient(endpoint=foundry_project_endpoint, credential=credential)
openai_client=project_client.get_openai_client()

prompt_content = "List the available smartphones in the catalog and place an order for one of each for me and my 10 friends."

# Load OpenAPI specifications
with open("./product-catalog-openapi.json", "r") as f:
    openapi_product_catalog = json.loads(f.read().replace("https://replace-me.local/catalogservice", f"{apim_resource_gateway_url}/catalogservice"))

with open("./place-order-openapi.json", "r") as f:
    openapi_place_order = json.loads(f.read().replace("https://replace-me.local/orderservice", f"{apim_resource_gateway_url}/orderservice"))

# Initialize agent OpenAPI tools using dictionary-based definitions
product_catalog_tool = {
    "type": "openapi",
    "openapi": {
        "name": "get_product_catalog",
        "description": "Retrieve the list of products available in the catalog",
        "spec": openapi_product_catalog,
        "auth": {
            "type": "project_connection",
            "security_scheme": {
                "project_connection_id": project_client.connections.get("ProductCatalogAPI").id
            }
        }
    }
}

place_order_tool = {
    "type": "openapi",
    "openapi": {
        "name": "place_order",
        "description": "Place a product order",
        "spec": openapi_place_order,
        "auth": {
            "type": "project_connection",
            "security_scheme": {
                "project_connection_id": project_client.connections.get("PlaceOrderAPI").id
            }
        }
    }
}

# Create agent with chained context managers
orders_agent = project_client.agents.create_version(
    agent_name="my-orders-agent",
    definition=PromptAgentDefinition(
        model=str(models_config[0].get('name')),
        instructions="You are a helpful sales assistant that helps users order products. Recover from errors if any and place multiple orders if needed.",
        tools=[product_catalog_tool, place_order_tool],
    ),
    description="You are a helpful sales assistant.",
)
utils.print_ok(f"Created agent (id: {orders_agent.id}, name: {orders_agent.name}, version: {orders_agent.version})")

conversation_id = openai_client.conversations.create().id
utils.print_ok(f"Created conversation, ID: {conversation_id}")

print(f"\n🗨️ User: {prompt_content}")
response = openai_client.responses.create(
    input=[{"role": "user", "content": prompt_content}],
    conversation=conversation_id,
    extra_body={"agent": {"name": orders_agent.name, "type": "agent_reference"}},
)
print(f"\n🗨️ Agent: {response.output_text}")

# print("\nCleaning up...")
# project_client.agents.delete_version(agent_name=orders_agent.name, agent_version=orders_agent.version)
# utils.print_ok("Agent deleted")

<a id='kql'></a>
### 🔍 Analyze Application Insights custom metrics with a KQL query

With this query you can get the custom metrics that were emitted by Azure APIM. Note that it may take a few minutes for data to become available.

In [ ]:
import pandas as pd

query = "\"" + "customMetrics \
| where name == 'Total Tokens' \
| where timestamp >= ago(1h) \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription ID']) \
| extend agentID = tostring(parsedCustomDimensions.['Agent ID']) \
| summarize TotalValue = sum(value) by apimSubscription, bin(timestamp, 1m), agentID \
| order by timestamp asc" + "\""

output = utils.run(f"az monitor app-insights query --app {app_insights_name} -g {resource_group_name} --analytics-query {query}",
    f"App Insights query succeeded", f"App Insights query  failed")

table = output.json_data['tables'][0]
df = pd.DataFrame(table.get("rows"), columns = [col.get("name") for col in table.get('columns')])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%H:%M')

df


<a id='plot'></a>
### 🔍 Plot the custom metrics results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [15, 7]
if df.empty:
    print("No data to plot")
else:
    df_pivot = df.pivot(index='timestamp', columns='apimSubscription', values='TotalValue')
    ax = df_pivot.plot(kind='bar', stacked=True)
    plt.title('Total token usage over time by APIM Subscription')
    plt.xlabel('Time')
    plt.ylabel('Tokens')
    plt.legend(title='APIM Subscription')
    plt.show()

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.